In [64]:
# %matplotlib widget

from __future__ import annotations


import matplotlib.colors as mcolors
from test_utilities import test, TestDict

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 12: Christmas Tree Farm ---</h2><p>You're almost out of time, but there can't be much left to decorate. Although there are no stairs, elevators, escalators, tunnels, chutes, teleporters, firepoles, or conduits here that would take you deeper into the North Pole base, there <em>is</em> a ventilation duct. You jump in.</p>
<p>After bumping around for a few minutes, you emerge into a large, well-lit cavern full of Christmas trees!</p>
<p>There are a few Elves here frantically decorating before the deadline. They think they'll be able to finish most of the work, but the one thing they're worried about is the <em>presents</em> for all the young Elves that live here at the North Pole. It's an ancient tradition to put the presents under the trees, but the Elves are worried they won't <em>fit</em>.</p>
<p>The presents come in a few standard but very weird shapes. The shapes and the regions into which they need to fit are all measured in standard <em>units</em>. To be aesthetically pleasing, the presents need to be placed into the regions in a way that follows a standardized two-dimensional unit grid; you also can't stack presents.</p>
<p>As always, the Elves have a summary of the situation (your puzzle input) for you. First, it contains a list of the presents' shapes. Second, it contains the size of the region under each tree and a list of the number of presents of each shape that need to fit into that region. For example:</p>
<pre><code>0:
###
##.
##.

1:

###

##.
.##

2:
.##

###

##.

3:
##.

###

##.

4:

###

#..

###

5:

###

.#.

###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2
</code></pre>

<p>The first section lists the standard present <em>shapes</em>. For convenience, each shape starts with its <em>index</em> and a colon; then, the shape is displayed visually, where <code>#</code> is part of the shape and <code>.</code> is not.</p>
<p>The second section lists the <em>regions</em> under the trees. Each line starts with the width and length of the region; <code>12x5</code> means the region is <code>12</code> units wide and <code>5</code> units long. The rest of the line describes the presents that need to fit into that region by listing the <em>quantity of each shape</em> of present; <code>1 0 1 0 3 2</code> means you need to fit one present with shape index 0, no presents with shape index 1, one present with shape index 2, no presents with shape index 3, three presents with shape index 4, and two presents with shape index 5.</p>
<p>Presents can be <em>rotated and flipped</em> as necessary to make them fit in the available space, but they have to always be placed perfectly on the grid. Shapes can't overlap (that is, the <code>#</code> part from two different presents can't go in the same place on the grid), but they <em>can</em> fit together (that is, the <code>.</code> part in a present's shape's diagram does not block another present from occupying that space on the grid).</p>
<p>The Elves need to know <em>how many of the regions</em> can fit the presents listed. In the above example, there are six unique present shapes and three regions that need checking.</p>
<p>The first region is 4x4:</p>
<pre><code>....
....
....
....
</code></pre>
<p>In it, you need to determine whether you could fit two presents that have shape index <code>4</code>:</p>
<pre><code>###
#..
###
</code></pre>
<p>After some experimentation, it turns out that you <em>can</em> fit both presents in this region. Here is one way to do it, using <code>A</code> to represent one present and <code>B</code> to represent the other:</p>
<pre><code>AAA.
ABAB
ABAB
.BBB
</code></pre>
<p>The second region, <code>12x5: 1 0 1 0 2 2</code>, is <code>12</code> units wide and <code>5</code> units long. In that region, you need to try to fit one present with shape index <code>0</code>, one present with shape index <code>2</code>, two presents with shape index <code>4</code>, and two presents with shape index <code>5</code>.</p>
<p>It turns out that these presents <em>can</em> all fit in this region. Here is one way to do it, again using different capital letters to represent all the required presents:</p>
<pre><code>....AAAFFE.E
.BBBAAFFFEEE
DDDBAAFFCECE
DBBB....CCC.
DDD.....C.C.
</code></pre>
<p>The third region, <code>12x5: 1 0 1 0 3 2</code>, is the same size as the previous region; the only difference is that this region needs to fit one additional present with shape index <code>4</code>. Unfortunately, no matter how hard you try, there is <em>no way to fit all of the presents</em> into this region.</p>
<p>So, in this example, <code><em>2</em></code> regions can fit all of their listed presents.</p>
<p>Consider the regions beneath each tree and the presents the Elves would like to fit into each of them. <em>How many of the regions can fit all of the presents listed?</em></p>
</article>


In [65]:
from pprint import pprint
import re

from more_itertools import split_at, spy
from tabulate import tabulate
from util import Str

tests: list[TestDict] = [
    {
        "name": "Example",
        "s": """
            0:
            ###
            ##.
            ##.

            1:
            ###
            ##.
            .##

            2:
            .##
            ###
            ##.

            3:
            ##.
            ###
            ##.

            4:
            ###
            #..
            ###

            5:
            ###
            .#.
            ###

            4x4: 0 0 0 0 2 0
            12x5: 1 0 1 0 2 2
            12x5: 1 0 1 0 3 2
        """,
        "expected": 2,
    },
]


class Summery(Str):
    # 9 wasn'tworking fortheexample, but did for both 8 is working for both :)
    # https://www.reddit.com/r/adventofcode/comments/1pkjynl/2025_day_12_day_12_solutions/#:~:text=First%2C%20pretend%20that%20every%20present,Undetermined.
    def __init__(self, s: str) -> None:
        *_, regions = re.split(r"\r?\n\s*\r?\n", s.strip())

        self.regions = [
            (r[:2], r[2:])
            for l in regions.splitlines()
            if (r := list(map(int, re.findall(r"\d+", l))))
        ]

    def fitting_regions(self) -> int:
        return sum(
            1
            for (width, height), presents in self.regions
            if 8 * sum(presents) < width * height
        )


@test(tests=tests)
def part_I(s: str) -> int:
    su = Summery(s)
    return su.fitting_regions()


Test Example passed, for part_I.
Success


In [66]:
with open("../input/day12.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 593


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>593</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<main>
<article>
<p>You help the Elves decorate the Christmas trees with all <span class="day-success">24 stars</span>! Now, the Elves will have plenty of time to prepare for Christmas, and you get a well-deserved break.</p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2025!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="https://was.tl/" target="_blank">my website</a> or through <a href="https://bsky.app/profile/was.tl" target="_blank">Bluesky</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2025/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://bsky.app/intent/compose?text=I+just+completed+all+12+days+of+Advent+of+Code+2025%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F" target="_blank">Bluesky</a>
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+12+days+of+Advent+of+Code+2025%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+12+days+of+Advent+of+Code+2025%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends or <a href="/2025">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf0" style="top:-1.09em; left:67.53vw; animation-delay:-10.25s;"></div><div class="sf sf3" style="top:-4.57em; left:64.58vw; animation-delay:-4.52s;"></div><div class="sf sf3" style="top:-4.11em; left:21.82vw; animation-delay:-0.63s;"></div><div class="sf sf2" style="top:-3.05em; left:4.97vw; animation-delay:-14.30s;"></div><div class="sf sf0" style="top:-2.00em; left:31.99vw; animation-delay:-15.78s;"></div><div class="sf sf1" style="top:-2.07em; left:51.82vw; animation-delay:-3.24s;"></div><div class="sf sf0" style="top:-1.38em; left:47.62vw; animation-delay:-16.22s;"></div><div class="sf sf2" style="top:-3.55em; left:64.88vw; animation-delay:-0.34s;"></div><div class="sf sf0" style="top:-1.23em; left:6.76vw; animation-delay:-8.21s;"></div><div class="sf sf1" style="top:-2.73em; left:5.23vw; animation-delay:-10.10s;"></div><div class="sf sf1" style="top:-2.83em; left:75.55vw; animation-delay:-19.16s;"></div><div class="sf sf1" style="top:-2.07em; left:56.75vw; animation-delay:-15.60s;"></div><div class="sf sf0" style="top:-1.11em; left:65.47vw; animation-delay:-15.19s;"></div><div class="sf sf2" style="top:-3.57em; left:73.91vw; animation-delay:-12.25s;"></div><div class="sf sf3" style="top:-4.46em; left:66.89vw; animation-delay:-13.14s;"></div><div class="sf sf1" style="top:-2.84em; left:72.93vw; animation-delay:-1.85s;"></div><div class="sf sf3" style="top:-4.73em; left:11.01vw; animation-delay:-8.37s;"></div><div class="sf sf1" style="top:-2.36em; left:19.41vw; animation-delay:-15.74s;"></div><div class="sf sf0" style="top:-1.70em; left:39.32vw; animation-delay:-13.20s;"></div><div class="sf sf3" style="top:-4.70em; left:57.50vw; animation-delay:-10.05s;"></div><div class="sf sf2" style="top:-3.52em; left:70.65vw; animation-delay:-0.95s;"></div><div class="sf sf0" style="top:-1.50em; left:78.59vw; animation-delay:-6.01s;"></div><div class="sf sf1" style="top:-2.91em; left:59.86vw; animation-delay:-15.68s;"></div><div class="sf sf0" style="top:-1.36em; left:22.21vw; animation-delay:-14.00s;"></div><div class="sf sf1" style="top:-2.67em; left:75.31vw; animation-delay:-18.84s;"></div><div class="sf sf2" style="top:-3.07em; left:36.49vw; animation-delay:-1.39s;"></div><div class="sf sf2" style="top:-3.62em; left:33.70vw; animation-delay:-0.29s;"></div><div class="sf sf3" style="top:-4.83em; left:12.56vw; animation-delay:-0.85s;"></div><div class="sf sf2" style="top:-3.93em; left:60.86vw; animation-delay:-3.87s;"></div><div class="sf sf0" style="top:-1.57em; left:94.29vw; animation-delay:-8.69s;"></div><div class="sf sf2" style="top:-3.30em; left:5.92vw; animation-delay:-5.08s;"></div><div class="sf sf0" style="top:-1.71em; left:48.62vw; animation-delay:-10.49s;"></div><div class="sf sf0" style="top:-1.53em; left:28.22vw; animation-delay:-12.03s;"></div><div class="sf sf0" style="top:-1.22em; left:16.32vw; animation-delay:-15.56s;"></div><div class="sf sf2" style="top:-3.42em; left:7.56vw; animation-delay:-0.09s;"></div><div class="sf sf0" style="top:-1.21em; left:3.80vw; animation-delay:-0.69s;"></div><div class="sf sf1" style="top:-2.19em; left:42.63vw; animation-delay:-15.03s;"></div><div class="sf sf0" style="top:-1.77em; left:93.56vw; animation-delay:-18.99s;"></div><div class="sf sf0" style="top:-1.98em; left:20.44vw; animation-delay:-10.15s;"></div><div class="sf sf0" style="top:-1.19em; left:73.57vw; animation-delay:-0.09s;"></div><div class="sf sf3" style="top:-4.48em; left:71.37vw; animation-delay:-18.43s;"></div><div class="sf sf1" style="top:-2.35em; left:75.31vw; animation-delay:-14.41s;"></div><div class="sf sf2" style="top:-3.88em; left:89.65vw; animation-delay:-2.82s;"></div><div class="sf sf0" style="top:-1.58em; left:39.98vw; animation-delay:-15.82s;"></div><div class="sf sf0" style="top:-1.15em; left:27.78vw; animation-delay:-14.47s;"></div><div class="sf sf2" style="top:-3.68em; left:33.90vw; animation-delay:-14.88s;"></div><div class="sf sf1" style="top:-2.79em; left:65.92vw; animation-delay:-1.97s;"></div><div class="sf sf0" style="top:-1.91em; left:26.36vw; animation-delay:-7.76s;"></div><div class="sf sf2" style="top:-3.97em; left:38.39vw; animation-delay:-12.84s;"></div><div class="sf sf1" style="top:-2.64em; left:26.48vw; animation-delay:-14.72s;"></div><div class="sf sf3" style="top:-4.32em; left:3.79vw; animation-delay:-12.75s;"></div><div class="sf sf2" style="top:-3.92em; left:45.57vw; animation-delay:-6.53s;"></div><div class="sf sf1" style="top:-2.29em; left:23.55vw; animation-delay:-17.57s;"></div><div class="sf sf1" style="top:-2.16em; left:1.47vw; animation-delay:-17.21s;"></div><div class="sf sf0" style="top:-1.69em; left:80.71vw; animation-delay:-8.71s;"></div><div class="sf sf1" style="top:-2.36em; left:38.78vw; animation-delay:-16.94s;"></div><div class="sf sf3" style="top:-4.42em; left:80.70vw; animation-delay:-3.26s;"></div><div class="sf sf2" style="top:-3.85em; left:91.07vw; animation-delay:-10.61s;"></div><div class="sf sf3" style="top:-4.30em; left:65.35vw; animation-delay:-16.74s;"></div><div class="sf sf2" style="top:-3.47em; left:35.50vw; animation-delay:-13.97s;"></div><div class="sf sf3" style="top:-4.49em; left:48.40vw; animation-delay:-8.90s;"></div><div class="sf sf0" style="top:-1.77em; left:29.98vw; animation-delay:-2.14s;"></div><div class="sf sf0" style="top:-1.70em; left:2.62vw; animation-delay:-0.46s;"></div><div class="sf sf0" style="top:-1.88em; left:89.40vw; animation-delay:-3.31s;"></div><div class="sf sf2" style="top:-3.31em; left:72.62vw; animation-delay:-12.33s;"></div><div class="sf sf3" style="top:-4.38em; left:74.19vw; animation-delay:-4.19s;"></div><div class="sf sf2" style="top:-3.99em; left:6.46vw; animation-delay:-7.92s;"></div><div class="sf sf3" style="top:-4.76em; left:48.27vw; animation-delay:-3.20s;"></div><div class="sf sf0" style="top:-1.58em; left:30.86vw; animation-delay:-13.76s;"></div><div class="sf sf3" style="top:-4.51em; left:22.46vw; animation-delay:-12.39s;"></div><div class="sf sf3" style="top:-4.69em; left:20.08vw; animation-delay:-14.15s;"></div><div class="sf sf2" style="top:-3.51em; left:94.92vw; animation-delay:-9.77s;"></div><div class="sf sf2" style="top:-3.27em; left:63.32vw; animation-delay:-1.34s;"></div><div class="sf sf1" style="top:-2.82em; left:91.99vw; animation-delay:-19.90s;"></div><div class="sf sf2" style="top:-3.50em; left:47.46vw; animation-delay:-14.40s;"></div><div class="sf sf3" style="top:-4.40em; left:75.05vw; animation-delay:-12.64s;"></div><div class="sf sf2" style="top:-3.12em; left:18.97vw; animation-delay:-17.18s;"></div><div class="sf sf3" style="top:-4.29em; left:13.25vw; animation-delay:-17.32s;"></div><div class="sf sf2" style="top:-3.52em; left:53.22vw; animation-delay:-8.62s;"></div><div class="sf sf2" style="top:-3.80em; left:21.32vw; animation-delay:-12.83s;"></div><div class="sf sf1" style="top:-2.63em; left:18.25vw; animation-delay:-6.84s;"></div><div class="sf sf2" style="top:-3.50em; left:47.97vw; animation-delay:-12.84s;"></div><div class="sf sf0" style="top:-1.92em; left:82.03vw; animation-delay:-3.07s;"></div><div class="sf sf2" style="top:-3.03em; left:32.16vw; animation-delay:-19.89s;"></div><div class="sf sf2" style="top:-3.84em; left:82.93vw; animation-delay:-0.61s;"></div><div class="sf sf0" style="top:-1.79em; left:57.88vw; animation-delay:-11.86s;"></div><div class="sf sf2" style="top:-3.03em; left:32.45vw; animation-delay:-19.64s;"></div><div class="sf sf0" style="top:-1.98em; left:62.07vw; animation-delay:-10.26s;"></div><div class="sf sf2" style="top:-3.94em; left:37.23vw; animation-delay:-19.78s;"></div><div class="sf sf2" style="top:-3.67em; left:90.68vw; animation-delay:-0.79s;"></div><div class="sf sf0" style="top:-1.43em; left:63.59vw; animation-delay:-8.51s;"></div><div class="sf sf2" style="top:-3.88em; left:14.43vw; animation-delay:-13.75s;"></div><div class="sf sf2" style="top:-3.38em; left:53.21vw; animation-delay:-6.06s;"></div><div class="sf sf2" style="top:-3.97em; left:83.15vw; animation-delay:-4.22s;"></div><div class="sf sf1" style="top:-2.41em; left:72.09vw; animation-delay:-5.14s;"></div><div class="sf sf0" style="top:-1.08em; left:76.08vw; animation-delay:-0.13s;"></div><div class="sf sf1" style="top:-2.03em; left:52.08vw; animation-delay:-17.33s;"></div><div class="sf sf0" style="top:-1.62em; left:10.60vw; animation-delay:-10.14s;"></div><div class="sf sf2" style="top:-3.98em; left:25.42vw; animation-delay:-16.92s;"></div><div class="sf sf0" style="top:-1.53em; left:22.10vw; animation-delay:-2.72s;"></div>
</main>


<link href="style.css" rel="stylesheet"></link>


In [67]:
# print(f"Part II: {part_II(puzzle)}")

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1474050600</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
